In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
import matplotlib as mpl
from jupyterthemes import jtplot #These two lines can be skipped if you are not using jupyter themes
jtplot.reset()

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.314)
import scipy as sp
import multiprocessing as mp

from matplotlib.ticker import StrMethodFormatter
import time
start_total = time.time()

In [2]:
from validity_class import sieplasma

In [3]:
import os
my_path = '/home/tomi/Documentos/Fisica/Tesis/escrito-tesis/images/'

In [4]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

In [5]:
zl = 0.2; zs = 1.2
Dl = cosmo.angular_diameter_distance(zl)   
Ds = cosmo.angular_diameter_distance(zs)    
Dls = cosmo.angular_diameter_distance_z1z2(zl, zs)
G = const.G
rho_crit = (cosmo.critical_density(zl)).to(u.kg/u.m**3)
c_light = (const.c).to(u.cm/u.second)

#r0 = 10*u.kpc
r0 = 10.0*u.kpc
#r0 = 0.1*u.kpc
pi = np.pi

def scale_radius(v,Dl,Ds,Dls):                               #this is e0 in eq 3.42 meneghetti, eq 1 barnacka 2014
    return (4.*pi*v**2/c_light**2*Dl*Dls/Ds).decompose()
def theta_E_SIS():
    'in arcsec'
    pre_theta_E = (scale_radius(v,Dl,Ds,Dls)/Dl).decompose()
    return pre_theta_E*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 

v = 180 *u.km/u.s
ss_r = scale_radius(v,Dl,Ds,Dls) 
print('scale radius (m): ',ss_r)
print('scale radius (kpc): ',ss_r.to(u.kpc))
print('theta_E: ',theta_E_SIS() ,'arcsec')
theta_E_num = theta_E_SIS()
re = (const.e.esu**2/const.m_e/(c_light**2)).decompose()
print('Classic electron radius: ',re)

scale radius (m):  7.705329461274929e+19 m
scale radius (kpc):  2.49712721364453 kpc
theta_E:  0.7301786792241515 arcsec
Classic electron radius:  2.817940324670788e-15 m


In [6]:
nu = 130.0 * u.MHz
omega = nu*(1.+zl)*2*np.pi
n0 = 300./(u.cm)**(3)

In [7]:
theta_0 = (r0/Dl).decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
print('Angular scale radius (theta_0 = r0/Dl): ',theta_0, 'arcsec')

Angular scale radius (theta_0 = r0/Dl):  2.9240748137875747 arcsec


In [8]:
from lenstronomy.LensModel.Profiles.plasma_exponential_sie import PlasmaExponentialSIE
perfil = PlasmaExponentialSIE()

In [9]:
def image_position_3(x, y, eta):
    x_s, y_s = x, y
    x_pre, y_pre = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, \
                    sourcePos_x=x_s, sourcePos_y=y_s, min_distance=0.05, search_window=5, \
                    precision_limit=10**(-8), num_iter_max=100,non_linear=True)
    print(x_pre)
    print(y_pre)
    mag = [0,0]
    t = [0,0]
    for k in range(len(x_pre)):
        mag[k] = lensModel.magnification(x_pre[k], y_pre[k], kwargs = kwargs_lens_list)
        phi = PlasmaExponentialSIE.function(perfil, x_pre[k], y_pre[k], theta_E_num.value, eta, A, B, C ,psi0_plasma.value, theta_0.value)
        t[k] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (x_s - x_pre[k] )**2 + (y_s - y_pre[k])**2) - phi**2)*(u.arcsec**2).to('rad**2')).to('s').value
    r = np.abs(mag[0]/mag[1])
    rr = [r,r]
    dt = np.abs(t[0] - t[1])/24/60/60 #convert seconds to days
    dt = [dt,dt]
    return x_pre, y_pre, rr, dt

In [10]:
lens_model_list = ['PlasmaExponentialSIE']
lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)
A = 2.00744300029693
B = 1.5887225542022732
C = 1.4308204791467736
psi0_plasma = (A*Dls/Ds/Dl*2*pi*c_light**2/omega**2*re*n0*r0 \
            *(u.rad**2).to('arcsec**2', equivalencies=u.dimensionless_angles()) ).decompose()
psi0_plasma_num = psi0_plasma.value
theta_E_g = theta_E_SIS().value
theta_0_num = theta_0.value

In [14]:
eta_arr = np.linspace(.05, .2, 50)
r_arr = np.linspace(.15, .3, 50)
x_arr = y_arr = r_arr*np.cos(np.pi/4)

In [15]:
result = []
for x in x_arr:
    for i in eta_arr:
        kwargs = {'theta_E':theta_E_num.value,'eta':i, 'A':A, 'B':B, 'C':C ,'psi0_plasma':psi0_plasma.value,\
                        'theta_0':theta_0.value}
        kwargs_lens_list = [kwargs]
        x_image, y_image, mag, dt = image_position_3(x, x, i)
        tmp = [x, i, mag[0], dt[0]]
        result.append(tmp)
        print(x,i)

[ 0.44014052 -0.44174463]
[ 0.69326445 -0.2779922 ]
0.10606601717798213 0.05
[ 0.43278685 -0.44450364]
[ 0.69843172 -0.27263725]
0.10606601717798213 0.05306122448979592
[ 0.42561413 -0.44709836]
[ 0.7033987  -0.26737806]
0.10606601717798213 0.056122448979591844
[ 0.41862357 -0.44953393]
[ 0.70817336 -0.26221809]
0.10606601717798213 0.05918367346938776
[ 0.41181525 -0.45181575]
[ 0.71276389 -0.25715997]
0.10606601717798213 0.06224489795918368
[ 0.40518834 -0.45394939]
[ 0.71717857 -0.25220562]
0.10606601717798213 0.0653061224489796
[ 0.39874115 -0.4559405 ]
[ 0.72142563 -0.24735632]
0.10606601717798213 0.06836734693877551
[ 0.39247134 -0.45779478]
[ 0.72551323 -0.24261276]
0.10606601717798213 0.07142857142857142
[ 0.38637601 -0.45951791]
[ 0.72944931 -0.2379751 ]
0.10606601717798213 0.07448979591836735
[ 0.3804518  -0.46111547]
[ 0.73324162 -0.23344308]
0.10606601717798213 0.07755102040816328
[ 0.37469498 -0.46259299]
[ 0.73689765 -0.22901605]
0.10606601717798213 0.08061224489795919
[ 0

[ 0.25388896 -0.46663142]
[ 0.81677778 -0.12895802]
0.10823062977345116 0.1846938775510204
[ 0.25158307 -0.46611936]
[ 0.81836975 -0.12701614]
0.10823062977345116 0.18775510204081636
[ 0.24932803 -0.46558363]
[ 0.81994005 -0.12511451]
0.10823062977345116 0.19081632653061226
[ 0.24712225 -0.4650251 ]
[ 0.82148958 -0.12325201]
0.10823062977345116 0.19387755102040816
[ 0.24496421 -0.4644446 ]
[ 0.8230192  -0.12142751]
0.10823062977345116 0.1969387755102041
[ 0.24285244 -0.46384292]
[ 0.82452973 -0.11963995]
0.10823062977345116 0.2
[ 0.44831719 -0.4348132 ]
[ 0.6949664  -0.27817543]
0.11039524236892018 0.05
[ 0.44109996 -0.43752152]
[ 0.70010829 -0.27301428]
0.11039524236892018 0.05306122448979592
[ 0.43405005 -0.44007572]
[ 0.70505964 -0.26793741]
0.11039524236892018 0.056122448979591844
[ 0.42716928 -0.44248012]
[ 0.70982737 -0.26294842]
0.11039524236892018 0.05918367346938776
[ 0.42045838 -0.44473928]
[ 0.71441868 -0.2580502 ]
0.11039524236892018 0.06224489795918368
[ 0.41391719 -0.4468

[ 0.28287031 -0.46353808]
[ 0.80589217 -0.14844093]
0.1125598549643892 0.1602040816326531
[ 0.28005514 -0.46328564]
[ 0.80772806 -0.14612719]
0.1125598549643892 0.163265306122449
[ 0.27730567 -0.46300016]
[ 0.80953219 -0.14386312]
0.1125598549643892 0.1663265306122449
[ 0.2746198  -0.46268294]
[ 0.811306   -0.14164733]
0.1125598549643892 0.16938775510204085
[ 0.27199551 -0.46233525]
[ 0.81305083 -0.13947847]
0.1125598549643892 0.17244897959183675
[ 0.26943083 -0.46195827]
[ 0.81476796 -0.13735521]
0.1125598549643892 0.17551020408163265
[ 0.26692389 -0.46155312]
[ 0.81645859 -0.1352763 ]
0.1125598549643892 0.1785714285714286
[ 0.26447289 -0.46112089]
[ 0.81812387 -0.13324049]
0.1125598549643892 0.1816326530612245
[ 0.26207607 -0.46066258]
[ 0.81976488 -0.13124658]
0.1125598549643892 0.1846938775510204
[ 0.25973177 -0.46017918]
[ 0.82138263 -0.12929342]
0.1125598549643892 0.18775510204081636
[ 0.25743836 -0.4596716 ]
[ 0.82297811 -0.12737987]
0.1125598549643892 0.19081632653061226
[ 0.25

[ 0.32019026 -0.45756652]
[ 0.79012837 -0.17382416]
0.11688908015532724 0.1326530612244898
[ 0.31664866 -0.45771656]
[ 0.79236483 -0.17101126]
0.11688908015532724 0.13571428571428573
[ 0.31319426 -0.45781805]
[ 0.79455178 -0.16825983]
0.11688908015532724 0.13877551020408163
[ 0.30982429 -0.45787303]
[ 0.79669161 -0.16556829]
0.11688908015532724 0.14183673469387756
[ 0.30653606 -0.45788344]
[ 0.79878651 -0.16293512]
0.11688908015532724 0.14489795918367349
[ 0.30332697 -0.45785112]
[ 0.80083862 -0.16035879]
0.11688908015532724 0.14795918367346939
[ 0.30019453 -0.45777781]
[ 0.80284991 -0.15783781]
0.11688908015532724 0.1510204081632653
[ 0.29713628 -0.45766518]
[ 0.80482227 -0.15537073]
0.11688908015532724 0.15408163265306124
[ 0.2941499  -0.45751482]
[ 0.80675749 -0.15295611]
0.11688908015532724 0.15714285714285714
[ 0.2912331  -0.45732824]
[ 0.80865723 -0.15059256]
0.11688908015532724 0.1602040816326531
[ 0.28838371 -0.45710686]
[ 0.8105231  -0.14827872]
0.11688908015532724 0.163265306

[ 0.36507918 -0.44691793]
[ 0.76949507 -0.20369352]
0.12121830534626529 0.10510204081632654
[ 0.36061404 -0.44763852]
[ 0.77234145 -0.20030765]
0.12121830534626529 0.10816326530612246
[ 0.35626111 -0.44828856]
[ 0.7751105  -0.19699308]
0.12121830534626529 0.11122448979591837
[ 0.35201727 -0.44887096]
[ 0.77780593 -0.19374857]
0.12121830534626529 0.1142857142857143
[ 0.34787943 -0.44938847]
[ 0.78043128 -0.19057281]
0.12121830534626529 0.11734693877551021
[ 0.34384452 -0.44984376]
[ 0.78298991 -0.18746445]
0.12121830534626529 0.12040816326530614
[ 0.33990953 -0.4502394 ]
[ 0.78548501 -0.18442213]
0.12121830534626529 0.12346938775510205
[ 0.3360715  -0.45057782]
[ 0.78791961 -0.18144444]
0.12121830534626529 0.12653061224489798
[ 0.33232753 -0.45086139]
[ 0.7902966  -0.17852999]
0.12121830534626529 0.12959183673469388
[ 0.32867478 -0.45109233]
[ 0.79261868 -0.17567734]
0.12121830534626529 0.1326530612244898
[ 0.32511048 -0.45127281]
[ 0.79488844 -0.17288508]
0.12121830534626529 0.13571428

...

[ 0.38695174 -0.35080592]
[ 0.88397711 -0.1405173 ]
0.19265052099674307 0.1969387755102041
[ 0.384399   -0.35057553]
[ 0.885854   -0.13883678]
0.19265052099674307 0.2
[ 0.5759723  -0.31954234]
[ 0.74575635 -0.2460092 ]
0.19481513359221206 0.05
[ 0.57073149 -0.32115529]
[ 0.75013008 -0.24343224]
0.19481513359221206 0.05306122448979592
[ 0.56553606 -0.32271886]
[ 0.75442672 -0.24085715]
0.19481513359221206 0.056122448979591844
[ 0.5603888  -0.32423287]
[ 0.75864611 -0.23828556]
0.19481513359221206 0.05918367346938776
[ 0.55529228 -0.32569723]
[ 0.76278828 -0.23571904]
0.19481513359221206 0.06224489795918368
[ 0.55024881 -0.32711191]
[ 0.7668535  -0.23315908]
0.19481513359221206 0.0653061224489796
[ 0.54526047 -0.32847698]
[ 0.77084218 -0.23060712]
0.19481513359221206 0.06836734693877551
[ 0.54032912 -0.32979256]
[ 0.77475494 -0.22806453]
0.19481513359221206 0.07142857142857142
[ 0.5354564  -0.33105887]
[ 0.77859253 -0.2255326 ]
0.19481513359221206 0.07448979591836735
[ 0.53064371 -0.3322

[ 0.40700091 -0.34583882]
[ 0.87734679 -0.14893993]
0.1969797461876811 0.1816326530612245
[ 0.4042238  -0.34573997]
[ 0.87936978 -0.14717334]
0.1969797461876811 0.1846938775510204
[ 0.4014912  -0.34561582]
[ 0.88136463 -0.14542825]
0.1969797461876811 0.18775510204081636
[ 0.39880225 -0.34546693]
[ 0.88333217 -0.14370446]
0.1969797461876811 0.19081632653061226
[ 0.39615613 -0.34529386]
[ 0.88527323 -0.14200178]
0.1969797461876811 0.19387755102040816
[ 0.39355199 -0.34509714]
[ 0.8871886 -0.14032  ]
0.1969797461876811 0.1969387755102041
[ 0.39098904 -0.34487729]
[ 0.88907905 -0.1386589 ]
0.1969797461876811 0.2
[ 0.58156721 -0.31430094]
[ 0.74895634 -0.24331086]
0.19914435878315012 0.05
[ 0.57639649 -0.31586592]
[ 0.75329206 -0.24081952]
0.19914435878315012 0.05306122448979592
[ 0.57126868 -0.31738395]
[ 0.75755376 -0.23832903]
0.19914435878315012 0.056122448979591844
[ 0.56618649 -0.31885482]
[ 0.76174118 -0.23584091]
0.19914435878315012 0.05918367346938776
[ 0.56115244 -0.32027841]
[ 0.

[ 0.43437421 -0.33998782]
[ 0.86538709 -0.16131614]
0.20130897137861914 0.1602040816326531
[ 0.43125964 -0.34008839]
[ 0.86764577 -0.1594266 ]
0.20130897137861914 0.163265306122449
[ 0.42819484 -0.3401599 ]
[ 0.86986971 -0.15755846]
0.20130897137861914 0.1663265306122449
[ 0.42517896 -0.34020294]
[ 0.87205995 -0.15571162]
0.20130897137861914 0.16938775510204085
[ 0.42221115 -0.34021813]
[ 0.87421748 -0.153886  ]
0.20130897137861914 0.17244897959183675
[ 0.41929059 -0.34020606]
[ 0.87634328 -0.15208149]
0.20130897137861914 0.17551020408163265
[ 0.41641642 -0.34016732]
[ 0.87843829 -0.15029796]
0.20130897137861914 0.1785714285714286
[ 0.41358781 -0.34010248]
[ 0.88050342 -0.14853528]
0.20130897137861914 0.1816326530612245
[ 0.41080393 -0.3400121 ]
[ 0.88253956 -0.14679331]
0.20130897137861914 0.1846938775510204
[ 0.40806397 -0.33989673]
[ 0.88454757 -0.14507188]
0.20130897137861914 0.18775510204081636
[ 0.40536711 -0.33975691]
[ 0.88652828 -0.14337084]
0.20130897137861914 0.1908163265306

[ 0.4572648  -0.33325641]
[ 0.85656461 -0.17020276]
0.2056381965695572 0.14489795918367349
[ 0.4538934  -0.33351327]
[ 0.85902372 -0.16824424]
0.2056381965695572 0.14795918367346939
[ 0.45057476 -0.33373858]
[ 0.86144288 -0.16630596]
0.2056381965695572 0.1510204081632653
[ 0.44730813 -0.33393295]
[ 0.86382322 -0.16438797]
0.2056381965695572 0.15408163265306124
[ 0.44409273 -0.33409698]
[ 0.86616585 -0.16249028]
0.2056381965695572 0.15714285714285714
[ 0.44092779 -0.33423128]
[ 0.86847185 -0.16061291]
0.2056381965695572 0.1602040816326531
[ 0.43781253 -0.33433645]
[ 0.87074229 -0.15875584]
0.2056381965695572 0.163265306122449
[ 0.43474615 -0.33441308]
[ 0.87297818 -0.15691903]
0.2056381965695572 0.1663265306122449
[ 0.43172786 -0.33446174]
[ 0.87518054 -0.15510244]
0.2056381965695572 0.16938775510204085
[ 0.42875685 -0.33448303]
[ 0.87735033 -0.15330599]
0.2056381965695572 0.17244897959183675
[ 0.42583232 -0.33447748]
[ 0.8794885  -0.15152962]
0.2056381965695572 0.17551020408163265
[ 0.

[ 0.49261016 -0.32427964]
[ 0.83830324 -0.18525232]
0.20996742176049527 0.12040816326530614
[ 0.48880862 -0.3248046 ]
[ 0.8411305  -0.18318582]
0.20996742176049527 0.12346938775510205
[ 0.48506332 -0.32529409]
[ 0.8439087  -0.18113683]
0.20996742176049527 0.12653061224489798
[ 0.48137381 -0.3257487 ]
[ 0.84663906 -0.17910563]
0.20996742176049527 0.12959183673469388
[ 0.47773962 -0.32616898]
[ 0.84932282 -0.1770925 ]
0.20996742176049527 0.1326530612244898
[ 0.47416021 -0.32655553]
[ 0.85196121 -0.17509765]
0.20996742176049527 0.13571428571428573
[ 0.47063502 -0.32690891]
[ 0.8545554 -0.1731213]
0.20996742176049527 0.13877551020408163
[ 0.46716344 -0.32722971]
[ 0.8571066  -0.17116361]
0.20996742176049527 0.14183673469387756
[ 0.46374484 -0.32751851]
[ 0.85961597 -0.16922474]
0.20996742176049527 0.14489795918367349
[ 0.46037856 -0.32777589]
[ 0.86208464 -0.16730479]
0.20996742176049527 0.14795918367346939
[ 0.45706392 -0.32800244]
[ 0.86451374 -0.16540387]
0.20996742176049527 0.151020408

In [16]:
np.shape(result)

(2500, 4)

In [20]:
result[1]

[0.10606601717798213,
 0.05306122448979592,
 1.580224513157333,
 11.89255848384349]

In [18]:
result_p = []
for x in x_arr:
    for i in eta_arr:
        sie_plasma = sieplasma(theta_E_g, i, zl, c_light.decompose().value, Dl.decompose().value, Ds.decompose().value, Dls.decompose().value, psi0_plasma_num, theta_0_num, B, C, 0.07, x, x)
        mag, dt = sie_plasma.R, sie_plasma.dt/24/60/60 
        tmp = [x, i, mag, dt]
        result_p.append(tmp)

In [21]:
result_p[1]

[0.10606601717798213,
 0.05306122448979592,
 1.0110239146460107,
 267861297891.4574]

In [23]:
from scipy.optimize import fsolve

In [24]:
eta = .05
deltab_10 = deltab_20 = .1
def g(phi_k):
    tmp_g =  theta_E_g*eta*np.sin(2.*phi_k)/np.sqrt(1.-eta*np.cos(2.*phi_k)) - deltab_10*np.sin(phi_k) + deltab_20*np.cos(phi_k)
    return tmp_g

phi_arr = np.array([np.pi/4, 5/4*np.pi])
zeros_g = fsolve(g, phi_arr)

In [25]:
zeros_g*180/np.pi

array([ 58.21681023, 211.53679168])

In [ ]:
end_total = time.time()
print('total time: ',(end_total-start_total)/60.,' minutes')